In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import pickle
import time

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# Settings
num_epochs = 10
batch_size = 32
dense_layers = [0] # [0, 1, 2] [0]
layer_sizes = [64] # [64] [32, 64, 128]
conv_layers = [3] # [3] [1,2,3]
activation_functions = ['relu', 'sigmoid']
optimiser = ['adam']



In [ ]:
# gpu_options = tf.config.gpu.set_per_process_memory_fraction(FRAC)
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sessiom = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)
y = np.array(y)
X = X/255.0

In [ ]:
for dense_layer in dense_layers:
    print(f"----Dense layer: {dense_layer}----")
    for layer_size in layer_sizes:
        print(f"----layer size: {layer_size}----")
        for conv_layer in conv_layers:
            print(f"----Conv layer: {conv_layer}----")
            NAME = f"d--{batch_size}-batch-{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            print(NAME)
            
            model = Sequential()
            
            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
                
            model.add(Flatten())
            
            for l in range(dense_layer):
                model.add(Dense(512)) 
                model.add(Activation("relu"))
                model.add(Dropout(0.2))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

            model.fit(X,y, batch_size=batch_size, epochs=num_epochs, validation_split=0.3, callbacks=[tensorboard])
model.save('64x3-CNN.model')